This exercise will require you to pull some data from https://data.nasdaq.com/ (formerly Quandl API).

As a first step, you will need to register a free account on the https://data.nasdaq.com/ website.

After you register, you will be provided with a unique API key, that you should store:

*Note*: Use a `.env` file and put your key in there and `python-dotenv` to access it in this notebook. 

The code below uses a key that was used when generating this project but has since been deleted. Never submit your keys to source control. There is a `.env-example` file in this repository to illusrtate what you need. Copy that to a file called `.env` and use your own api key in that `.env` file. Make sure you also have a `.gitignore` file with a line for `.env` added to it. 

The standard Python gitignore is [here](https://github.com/github/gitignore/blob/master/Python.gitignore) you can just copy that. 

In [3]:
!pip install python-dotenv

In [163]:
# get api key from your .env file
import os
from dotenv import load_dotenv  # if missing this module, simply run `pip install python-dotenv`

load_dotenv()
API_KEY = os.getenv('NASDAQ_API_KEY')

print(API_KEY)

Nasdaq Data has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Nasdaq Data API instructions here: https://docs.data.nasdaq.com/docs/in-depth-usage

While there is a dedicated Python package for connecting to the Nasdaq API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [154]:
# First, import the relevant modules
import collections
import requests
import json

Note: API's can change a bit with each version, for this exercise it is reccomended to use the nasdaq api at `https://data.nasdaq.com/api/v3/`. This is the same api as what used to be quandl so `https://www.quandl.com/api/v3/` should work too.

Hint: We are looking for the `AFX_X` data on the `datasets/FSE/` dataset.

In [155]:
# Now, call the Nasdaq API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned

database_code = 'FSE'
dataset_code = 'AFX_X'
param = {
    'start_date': '2000-06-07', 
    'end_date': '2000-06-07'
}

url = f"https://data.nasdaq.com/api/v3/datasets/{database_code}/{dataset_code}/data.json?api_key={API_KEY}"
r = requests.get(url, params=param)
if r.ok:
    data = json.loads(r.content)
else:
    print(r.raise_for_status())

In [156]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure
data

{'dataset_data': {'limit': None,
  'transform': None,
  'column_index': None,
  'column_names': ['Date',
   'Open',
   'High',
   'Low',
   'Close',
   'Change',
   'Traded Volume',
   'Turnover',
   'Last Price of the Day',
   'Daily Traded Units',
   'Daily Turnover'],
  'start_date': '2000-06-07',
  'end_date': '2000-06-07',
  'frequency': 'daily',
  'data': [['2000-06-07',
    None,
    None,
    None,
    38.0,
    None,
    0.0,
    None,
    None,
    None,
    0.0]],
  'collapse': None,
  'order': None}}

These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

In [157]:
# 1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
# 2. Convert the returned JSON object into a Python dictionary.
database_code = 'FSE'
dataset_code = 'AFX_X'
param = {
    'start_date': '2017-01-01', 
    'end_date': '2017-12-31'
}

url = f"https://data.nasdaq.com/api/v3/datasets/{database_code}/{dataset_code}/data.json?api_key={API_KEY}"
r = requests.get(url, params=param)

if r.ok:
    data_afxx_2017 = json.loads(r.content)
else:
    print(r.raise_for_status())

In [82]:
data_afxx_2017

{'dataset_data': {'limit': None,
  'transform': None,
  'column_index': None,
  'column_names': ['Date',
   'Open',
   'High',
   'Low',
   'Close',
   'Change',
   'Traded Volume',
   'Turnover',
   'Last Price of the Day',
   'Daily Traded Units',
   'Daily Turnover'],
  'start_date': '2017-05-01',
  'end_date': '2017-05-01',
  'frequency': 'daily',
  'data': [['2017-05-01',
    None,
    42.245,
    41.655,
    41.72,
    -0.44,
    86348.0,
    3606589.0,
    None,
    None,
    None]],
  'collapse': None,
  'order': None}}

In [158]:
# 3. Calculate what the highest and lowest opening prices were for the stock in this period.

#index of Open
open_ndx = 1

# init values
highest = float('-inf')
lowest = float('inf')

# get data
data = data_afxx_2017['dataset_data'].get('data')

# extract max, min opening prices
for dataum in data:
    if dataum[open_ndx] != None:
        if highest < dataum[open_ndx]:
            highest = dataum[open_ndx]
        if lowest > dataum[open_ndx]:
            lowest = dataum[open_ndx]
    """
    # highest and lowest prices in general
    if highest < dataum[high_ndx]:
        highest = dataum[high_ndx]
    if lowest > dataum[low_ndx]:
        lowest = dataum[low_ndx]
    """
# in case all data in Open are None then raise error
if highest == float('-inf') or lowest == float('inf'):
    raise Exception("No data found")

# result
print(f"highest: {highest}, lowest: {lowest}")

highest: 53.11, lowest: 34.0


In [159]:
# 4. What was the largest change in any one day (based on High and Low price)?

#indexes of High and Low
high_ndx = 2
low_ndx = 3

largest_change = float('-inf')

# get data
data = data_afxx_2017['dataset_data'].get('data')

for dataum in data:
    if dataum[high_ndx] != None and dataum[low_ndx] != None:
        if largest_change < dataum[high_ndx] - dataum[low_ndx]:
            largest_change = dataum[high_ndx] - dataum[low_ndx]

# in case all data in Open are None then raise error
if largest_change == float('-inf'):
    raise Exception("No data found")

# result
print(f"largest_change: {largest_change}")

largest_change: 2.8100000000000023


In [160]:
# 5. What was the largest change between any two days (based on Closing Price)?

# any two days means any two random days. 
# require to have at least two days with out missing readings. two solutions: 
# (1) we calculate day 0 with all following days, then day 1 with all following days, etc. 
# (2) we calculate the highest and lowest closing and get the change (faster). 
# note: solution will be different if the question said "any two consecutive days"

#indexes of Close
close_ndx = 4

largest_change = float('-inf')

# get data
data = data_afxx_2017['dataset_data'].get('data')


"""
# 1st attempt
if len(data) > 2:
    for selected_day in range(len(data)):
        day_1_closing = data[selected_day][close_ndx]
        day_1_date = data[selected_day][0]

        for dataum in data[selected_day+1:]:
            if dataum[close_ndx] != None:
                day_2_closing = dataum[close_ndx]

                if largest_change < day_1_closing - day_2_closing:
                    largest_change = day_1_closing - day_2_closing
                    day_2_date = dataum[0]
else:
    print(f"total data:{len(data)} is not enough to calculate the change")
    
"""

# 2nd attempt

highest_closing = float('-inf')
lowest_closing = float('inf')

if len(data) > 1:
    for dataum in data:
        
        if highest_closing < dataum[close_ndx]:
            highest_closing = dataum[close_ndx]
            day_1_date = dataum[0]
            
        if lowest_closing > dataum[close_ndx]:
            lowest_closing = dataum[close_ndx]
            day_2_date = dataum[0]
    
    largest_change = highest_closing - lowest_closing
    
# in case all data in Open are None then raise error
if largest_change == float('-inf'): 
    print("No data found")
else:
    # result
    print(f"largest_change is between day:{day_1_date}({highest_closing}) and day:{day_2_date}({lowest_closing}) where the change is {largest_change}")

largest_change is between day:2017-12-13(53.09) and day:2017-01-23(34.06) where the change is 19.03


In [161]:
# 6. What was the average daily trading volume during this year?

#indexes of Close
traded_vol_ndx = 6
days, total = 0, 0

# get data
data = data_afxx_2017['dataset_data'].get('data')

if len(data) > 0:
    for dataum in data:
        # will exclude the day when data is missing
        if dataum[traded_vol_ndx] != None:
            days += 1
            total += dataum[traded_vol_ndx]
    avg = total/days
    print(f"average daily trading volume during 2017 for {days} days is {avg}")
else:
    print("No data found")
    avg = 0

average daily trading volume during 2017 for 255 days is 89124.33725490196


In [162]:
# 7.(Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

traded_vol_ndx = 6
traded_list = []
median = None 

# get data
data = data_afxx_2017['dataset_data'].get('data')

if len(data) > 0:
    for dataum in data:
        traded_list.append(dataum[traded_vol_ndx])
    traded_list.sort()
    median = (traded_list[(len(traded_list)//2)] + traded_list[(len(traded_list)//2)-1])/2 if len(traded_list)%2 == 0 else traded_list[len(traded_list)//2]
    print(f"the median trading volume during this year is {median}")
else:
    print("No data found")

the median trading volume during this year is 76286.0
